<a href="https://colab.research.google.com/github/sallumandya1995/datascience/blob/master/assemblyai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
 

Load the pipeline

 **Cohere one shot lerning capbility**
 
 
Using power of Cohere's command model to write a detailed caption of an image describing the interior design of living room and letting stable diffusion do the remaining magic
 

In [ ]:
#@title init libraries for GPu
!pip install -qq cohere
!pip install -qq gradio
!pip install -qq accelerate
!pip install -qq diffusers==0.8.0   ftfy
!pip install -qq "ipywidgets>=7,<8"
 
!pip install git+https://github.com/huggingface/transformers
import inspect
import warnings
from typing import List, Optional, Union

import torch
from torch import autocast
from tqdm.auto import tqdm

from diffusers import StableDiffusionImg2ImgPipeline
  
import random
import inspect
import warnings
from typing import List, Optional, Union

import torch
from torch import autocast
from tqdm.auto import tqdm

from diffusers import StableDiffusionImg2ImgPipeline

import requests
import os
 
from IPython.display import Image, display
import cohere

from huggingface_hub import notebook_login
import inspect
import warnings
from typing import List, Optional, Union

import torch
from torch import autocast
from tqdm.auto import tqdm

from diffusers import StableDiffusionImg2ImgPipeline
import inspect
import warnings
from typing import List, Optional, Union
 

 

!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('xxxxxxxxxxxxxxxx')" #pass huggingface token as arguement

 
co = cohere.Client('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')   #use your cohere api inside

os.system("gdown https://drive.google.com/uc?id=1mT9ewx86PSrc43b-ax47l1E2UzR7Ln4j -O RealESRGAN_x8.pth")
device = "cuda"
model_path = "prompthero/midjourney-v4-diffusion"
model_path="stabilityai/stable-diffusion-2"

model_path = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
#     revision="fp16",  # comment for prompt hero version
    torch_dtype=torch.float16
    ,use_auth_token=True
)
pipe = pipe.to(device)


#upscalers

!git clone https://huggingface.co/spaces/Xhaheen/Face-Real-ESRGAN
!pip install -q gdown gradio
!cp -r Face-Real-ESRGAN/. .

# %cd Face-Real-ESRGAN

In [ ]:
#@title using cohere  and gpus
import csv
from PIL import Image


import re
import datetime


 
import random
import torch
from PIL import Image
import numpy as np
from realesrgan import RealESRGAN
import os
import gradio as gr
import os

import matplotlib.pyplot as plt
!wget https://huggingface.co/spaces/Xhaheen/Baith-al-suroor/resolve/main/1.png
!wget https://huggingface.co/spaces/Xhaheen/Face-Real-ESRGAN/resolve/main/RealESRGAN_x8.pth 
if not os.path.exists("RealESRGAN_x8.pth"):
  !wget https://huggingface.co/spaces/Xhaheen/Face-Real-ESRGAN/resolve/main/RealESRGAN_x8.pth

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


        
current_time = datetime.datetime.now()
def generate_caption_keywords(prompt, model='command-xlarge-20221108', max_tokens=200, temperature=random.uniform(0.1, 2), k=0, p=0.75, frequency_penalty=0, presence_penalty=0, stop_sequences=[]):
    
    response = co.generate(
      model=model,
      prompt=prompt,
      max_tokens=max_tokens,
      temperature=temperature,
      k=k,
      p=p,
      frequency_penalty=frequency_penalty,
      presence_penalty=presence_penalty,
      stop_sequences=stop_sequences,
      return_likelihoods='NONE')

    def highlight_keywords(text):
        keywords = []
        text = text.lower()
        text = re.sub(r'[^a-z\s]', '', text) # remove punctuation
        text = re.sub(r'\b(the|and|of)\b', '', text) # remove stop words
        words = text.split()
        for word in words:
            if word not in keywords:
                keywords.append(word)
        return keywords

    caption = response.generations[0].text
    keywords = highlight_keywords(caption)
    keywords_string = ', '.join(keywords)

    return caption, keywords_string


def img2img( path ,is_HD,design,x_prompt,alt_prompt,strength,guidance_scale,steps):
  current_time = datetime.datetime.now()
  prompt = f'write a detailed caption of an image describing the  {design}'
  
  try:
        caption, keywords = generate_caption_keywords(prompt)
        prompt = keywords
  except:
    
    prompt = design
    

  if x_prompt == True:
    
    prompt=alt_prompt
    print(alt_prompt ,strength,7.9,current_time)
     
 
  generator = torch.Generator(device=device).manual_seed(1024) 
  
 
    
  init_img =Image.open(path)
#   init_img.save(f"original_{prompts[0]}_strength{strength}_guidance_{guidance_scale}.jpeg")

  with autocast("cuda"):
        
      
      image2 = pipe(prompt=prompt , init_image=init_img, strength=strength,num_inference_steps=steps, guidance_scale=guidance_scale, generator=generator).images[0]
      
 #remove if condition in above line if it does not fits
      if is_HD == True:


        model8 = RealESRGAN(device, scale=8)

         
        model8.load_weights('RealESRGAN_x8.pth')
        image2 = model8.predict(image2.convert('RGB'))
        
 
      image2.save("image2.jpeg")
      image2.save(f"new_{current_time}{design}_strength{strength}_guidance_{guidance_scale}.jpeg")
#       print(design,prompt ,strength,7.9,current_time)
      # Import the pyplot module from Matplotlib

 

      # Display the image using the imshow function
      plt.imshow(image2)

      # Show the plot on the screen
      plt.show()

      print(f"new_{current_time}{design}_strength{strength}_guidance_{guidance_scale}.jpeg")
      current_image=f"new_{current_time}{design}_strength{strength}_guidance_{guidance_scale}.jpeg"
      


      # Download the file # comment for kaggle
      from google.colab import files
      files.download(f"new_{current_time}{design}_strength{strength}_guidance_{guidance_scale}.jpeg")
 
      
  def append_csv(filename, row):
      with open(filename, "a") as f:
          writer = csv.writer(f)
          writer.writerow(row)
# records the data bases to store inputs and outputs for future reference
  append_csv("database.csv", [current_time,strength,guidance_scale,f"new_{prompt}_strength{strength}_guidance_{guidance_scale}.jpeg"])

  
  return image2

 